## IMPORT LIBRARIES AND SETTINGS

In [1]:
%load_ext autoreload
%matplotlib inline
%autoreload 2

In [2]:
from IPython import display
from utils import Logger

import torch
from torch import nn,optim
from torch.autograd.variable import Variable
from torchvision import transforms,datasets

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


## DATA LOADING

In [3]:
DATA_FOLDER_PATH = './torch_data/VGAN/MNIST'

In [4]:
compose     = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
out_dir     = '{}/dataset'.format(DATA_FOLDER_PATH)
batch_size  = 100
data        = datasets.MNIST(root=out_dir, train=True, transform=compose, download=True)
data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)
num_batches = len(data_loader)

## DISCRIMINATOR

In [5]:
class DiscriminatorNet(torch.nn.Module):
    """
    A three hidden-layer discriminative neural network
    """
    def __init__(self):
        super(DiscriminatorNet,self).__init__()
        n_features = 28*28
        n_out = 1
        
        self.hidden0 = nn.Sequential( 
            nn.Linear(n_features, 1024),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.hidden1 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.hidden2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.out = nn.Sequential(
            torch.nn.Linear(256, n_out),
            torch.nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.out(x)
        return x

## GENERATOR

In [6]:
def images_to_vectors(images):
    return images.view(images.size(0),28*28)

def vectors_to_images(vector):
    return vector.view(vector.size(0),1,28,28)

class GeneratorNet(torch.nn.Module):
    """
    A three hidden-layer generative neural network
    """
    def __init__(self):
        super(GeneratorNet,self).__init__()

        n_features = 100
        n_out = 28*28

        self.hidden0 = nn.Sequential(
                nn.Linear(n_features, 256),
                nn.LeakyReLU(0.2)
            )
        self.hidden1 = nn.Sequential(            
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2)
        )
        self.hidden2 = nn.Sequential(
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2)
        )

        self.out = nn.Sequential(
            nn.Linear(1024, n_out),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.out(x)
        return x

def noise(size):
    noise = Variable(torch.randn(size,batch_size))
    if torch.cuda.is_available(): 
        return noise.cuda()
    return noise

In [7]:
discriminator = DiscriminatorNet()
generator = GeneratorNet()

if torch.cuda.is_available():
    discriminator.cuda()
    generator.cuda()

## OPTIMIZATION

In [8]:
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002)
g_optimizer = optim.Adam(generator.parameters(), lr=0.0002)

loss = nn.BCELoss()

discriminator_steps = 1
num_epochs = 200

## TRAINING

In [9]:
def real_data_target(size):
    '''
    Tensor containing ones, with shape = size
    '''
    data = Variable(torch.ones(size, 1))
    if torch.cuda.is_available(): 
        return data.cuda()
    return data

def fake_data_target(size):
    '''
    Tensor containing zeros, with shape = size
    '''
    data = Variable(torch.zeros(size, 1))
    if torch.cuda.is_available(): 
        return data.cuda()
    return data

In [10]:
def train_discriminator(optimizer, real_data, fake_data):
    
    optimizer.zero_grad()  # Reset gradients
    
    prediction_real = discriminator(real_data)
    error_real = loss(prediction_real,real_data_target(prediction_real.size(0)))
    error_real.backward()
    
    prediction_fake = discriminator(fake_data)
    error_fake = loss(prediction_fake,fake_data_target(prediction_fake.size(0)))
    error_fake.backward()
    
    optimizer.step()
    
    return error_real + error_fake, prediction_real, prediction_fake

In [11]:
def train_generator(optimizer, fake_data):
    
    optimizer.zero_grad()
    prediction = discriminator(fake_data)
    
    error = loss(prediction, real_data_target(prediction.size(0)))
    error.backward()
    
    optimizer.step()
    
    return error

## SAMPLES FOR TEST

In [12]:
num_test_samples = 16
test_noise = noise(num_test_samples)

## START OF TRAINING

In [13]:
logger = Logger(model_name='VGAN', data_name='MNIST')

for epoch in range(num_epochs):
    for n_batch, (real_batch,_) in enumerate(data_loader):

        
        real_data = Variable(images_to_vectors(real_batch))
        if torch.cuda.is_available(): real_data = real_data.cuda()
        
        fake_data = generator(noise(real_data.size(0))).detach()
        
        d_error, d_pred_real, d_pred_fake = train_discriminator(d_optimizer,real_data, fake_data)

        
        fake_data = generator(noise(real_batch.size(0)))
        
        g_error = train_generator(g_optimizer, fake_data)
        
        logger.log(d_error, g_error, epoch, n_batch, num_batches)

        
        if (n_batch) % 100 == 0:
            display.clear_output(True)
            
            test_images = vectors_to_images(generator(test_noise)).data.cpu()
            logger.log_images(test_images, num_test_samples, epoch, n_batch, num_batches);
            # Display status Logs
            logger.display_status(
                epoch, num_epochs, n_batch, num_batches,
                d_error, g_error, d_pred_real, d_pred_fake
            )
        # Model Checkpoints
        logger.save_models(generator, discriminator, epoch)

AttributeError: 'function' object has no attribute 'Variable'